**why i dropped other two csvs**
- The core objective of this project is collaborative filtering, which relies on user–item rating interactions, not textual metadata.
- Incorporating tags would require natural language processing or content-based feature engineering, which is outside the primary modeling focus of this project.
- Using tags is optional, not required, and does not improve the evaluation of collaborative filtering performance in this context.

- The links dataset was excluded because it contains only external identifiers and does not provide features relevant to modeling user preferences or improving recommendation quality
- As such, it was not necessary for the collaborative filtering task


In [30]:
#loading the datasets
import pandas as pd

ratings = pd.read_csv("data/ratings.csv")
movies = pd.read_csv("data/movies.csv")

In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [8]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9742 non-null   int64 
 1   title    9742 non-null   object
 2   genres   9742 non-null   object
dtypes: int64(1), object(2)
memory usage: 228.5+ KB


No missing values were found for both the two datasets

In [12]:
ratings = ratings.drop(columns=["timestamp"])

Timestamp was not required for rating-based collaborative filtering in this project.

In [ ]:
#Filter sparse users and movies
user_counts = ratings["userId"].value_counts()
ratings = ratings[ratings["userId"].isin(user_counts[user_counts >= 5].index)] #in here i kept only users with at least 5 ratings

movie_counts = ratings["movieId"].value_counts()
ratings = ratings[ratings["movieId"].isin(movie_counts[movie_counts >= 5].index)] #at least 5 ratings per movie

In [17]:
#Confirm the join key exists and matches
ratings["movieId"].dtype
movies["movieId"].dtype

dtype('int64')

In [ ]:
#both datasets have the same dtype, int64, for the join key, hence they are compatible for merging

In [22]:
#merge
df = ratings.merge(
    movies,
    on="movieId",
    how="left"
)

In [23]:
df.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90274 entries, 0 to 90273
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   userId   90274 non-null  int64  
 1   movieId  90274 non-null  int64  
 2   rating   90274 non-null  float64
 3   title    90274 non-null  object 
 4   genres   90274 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 3.4+ MB


In [27]:
df.describe()

,userId,movieId,rating
count,90274.000000,90274.000000,90274.000000
mean,321.462104,16284.192879,3.537358
std,181.825689,31375.602906,1.029858
min,1.000000,1.000000,0.500000
25%,171.000000,1084.000000,3.000000
50%,318.000000,2692.000000,3.500000
75%,474.000000,6711.000000,4.000000
max,610.000000,187595.000000,5.000000
